# soft clustering for genes --R
# (data of pituitary as an example) 

## library packages and load data

In [ ]:
library(Seurat)
library(Matrix)
library(stringr)
library(SeuratDisk)
library(Mfuzz)
library(dplyr)

#detach("package:monocle3")

In [4]:
work_dir = "scPhaSeMod/"
setwd(work_dir)

In [2]:
scRNA1 = readRDS('/data/pituitary.rds')

In [4]:
scRNA1@meta.data$cell_type %>% as.vector() %>% unique() %>% sort() -> celltypes

celltypes
print(length(celltypes))

[1] "CC"           "Corticotrope" "EC"           "Gonadotrope"  "Imm"         
 [6] "Lactotrope"   "MC"           "PP"           "Pre.Gonado"   "Pro.PIT1"    
[11] "RBC"          "Somatotrope"  "Stem"         "Thyrotrope"

[1] 14


In [ ]:
for(i in 1:length(celltypes)){
    celltype <- celltypes[i]
    if(!dir.exists(path=paste0("./gene_cluster/",celltype,'/'))){
        dir.create(path=paste0("./gene_cluster/",celltype,'/'))
    }
}

## soft clustering and draw expression trends

In [14]:
mfuzz_analysis <- function(celltype,cluster_num,showDF=TRUE,save=FALSE,showPic=TRUE){
    mean_raw <- read.csv(paste0("./data/expression_mean/timepoint_mean_data_",celltype,".csv"),row.names=1,check.names = FALSE)
    vars <- VariableFeatures(scRNA1) %>% as.vector()
    mean_expr <- mean_raw[vars,][order(vars),]
    if(showDF){
        mean_expr %>% head(., 3) %>% print()
        dim(mean_expr) %>% print()}
    options(repr.plot.width=7, repr.plot.height=7)
    mfuzz_ <- as.matrix(mean_expr)
    mfuzz_class <- new('ExpressionSet',exprs = mfuzz_)
    
    options(repr.plot.height=7,repr.plot.width=7)
    #预处理缺失值或者异常值
    mfuzz_class <- filter.NA(mfuzz_class, thres = 0.25)
    mfuzz_class <- fill.NA(mfuzz_class, mode = 'knn') 
    mfuzz_class <- filter.std(mfuzz_class, min.std = 0)

    mfuzz_class <- standardise(mfuzz_class)

    set.seed(321)
    mest <- mestimate(mfuzz_class)
    mfuzz_cluster <- mfuzz(mfuzz_class, c = cluster_num, m = mest)
    mfuzz_cluster$size %>% print()
    
    color<-c('#F2EFF8','#F2EFF8','#EAE1F2','#EFD8E0','#90A4AE')
    if(showPic){
        options(repr.plot.width=15, repr.plot.height=20)
        mfuzz.plot2(mfuzz_class, cl = mfuzz_cluster, x11=FALSE, mfrow = c(4,2),time.labels = colnames(mfuzz_), 
                    cex.axis = 2, cex.lab = 2, cex.main = 2, centre=TRUE, centre.lwd=5,
                    centre.col = '#b56a6f',colo=color)
    }
    if(save){
        paste0("./figures/",celltype,"_cl",cluster_num,".pdf") %>% pdf(width = 20,height = 30)
        options(repr.plot.width=15, repr.plot.height=20)
        mfuzz.plot2(mfuzz_class, cl = mfuzz_cluster, x11=FALSE, mfrow = c(4,2),time.labels = colnames(mfuzz_), 
            cex.axis = 2, cex.lab = 2, cex.main = 2, centre=TRUE, centre.lwd=5, centre.col = '#b56a6f',colo=color)
        dev.off()
    }
    
    mfuzz_result <- vector('list',length=3)
    mfuzz_result[[1]] <- mfuzz_
    mfuzz_result[[2]] <- mfuzz_cluster
    mfuzz_result[[3]] <- mfuzz_class
    return(mfuzz_result)
}

## geneorder within each gene cluster 

In [15]:
mfuzz_geneOrder <- function(mfuzz_result,celltype,pearson.prop,showDF=TRUE,save=FALSE){
    mfuzz_ <- mfuzz_result[[1]]
    mfuzz_cluster <- mfuzz_result[[2]]
    mfuzz_class <- mfuzz_result[[3]]
    
    mfuzz_cluster_results <- as.data.frame(mfuzz_cluster$cluster)
    colnames(mfuzz_cluster_results) <- c("mfuzz_cluster")

    mfuzz_center_score <- mfuzz_cluster$membership %>% as.data.frame()
    colnames(mfuzz_center_score) <- paste("cluster_",(colnames(mfuzz_center_score) %>% as.vector()))
    mfuzz_center_score$cluster <- mfuzz_cluster_results$mfuzz_cluster %>% as.vector()
    if(showDF){
        print("mfuzz_center_score:")
        head(mfuzz_center_score,3) %>% print()}

    cluster <- mfuzz_cluster$cluster
    cluster <- cbind(mfuzz_[names(cluster), ], cluster)
    cluster <- as.data.frame(cluster)
    if(showDF){
        print("cluster:")
        head(cluster,3) %>% print()}

    standard_expr <- mfuzz_class@assayData$exprs %>% as.data.frame()
    standard_expr["cluster"] <- cluster$cluster %>% as.vector()
    if(showDF){
        print("standard_expr:")
        head(standard_expr,3) %>% print()}
    centers_ <- mfuzz_cluster$centers %>% as.data.frame()
    cluster_num <- length(colnames(mfuzz_center_score)) - 1
    if(showDF){
        print("centers:")
        centers_ %>% print()}

    vector_list <- vector("list", length = dim(centers_)[1])
    vector_list_cluster <- vector("list", length = dim(centers_)[1])
    for (i in 1:length(vector_list)) {
        Centers1 <- centers_[rownames(centers_) == i, ]
        t(as.matrix(Centers1)) %>% as.data.frame() -> vector_list[[i]]

        Centers2 <- standard_expr[standard_expr$cluster ==i, -dim(standard_expr)[2] ]
        Centers2 <- as.matrix(Centers2) %>% t() %>% as.data.frame() -> vector_list_cluster[[i]]
    }

    final_result <- data.frame()
    for (x in 1:length(vector_list)) {
        result <- c()
        for (i in 1:length(colnames(vector_list_cluster[[x]]))){
            result <- c(result, cor(vector_list_cluster[[x]][, i], vector_list[[x]][,1],
                            method = "pearson") %>% round(., 2))
            # pearson correlation as a score for centers
        }
        df_pearson <- data.frame(pearson = result,
                                 gene = colnames(vector_list_cluster[[x]]),
                                 cluster = rep(x,length(result)))
        df_pearson <- df_pearson[order(df_pearson$pearson, decreasing = T),]
        final_result <- rbind(final_result, df_pearson)
    }
    if(showDF){
        print("final_result:")
        head(final_result,3) %>% print()}

    plot(pearson~cluster, data=final_result)
    boxplot(pearson~cluster,data=final_result)

    final_result %>% group_by(cluster) %>%
    slice_max(., order_by = pearson, prop = pearson.prop) %>% as.data.frame() -> tmp
    print("Table of clusters:")
    tmp$cluster %>% table() %>% print()
    plot(pearson~cluster ,data=tmp)
    paste0("Pearson Prop: ",pearson.prop,"    The minimal pearson correlation: ",min(tmp$pearson)) %>% print()
    if(save){
        write.csv(final_result,paste0('.data/gene_cluster/',celltype,'/',celltype,'_GeneOrder_origin.csv'))
        paste0('.data/gene_cluster/',celltype,'/',celltype,'_GeneOrder_origin.csv') %>% print()
    }
    return(final_result)
}

## run and output

In [ ]:
for(j in 1:length(celltypes)){
    celltype <- celltypes[j]
    print(celltype)
    cn = 16
    mfuzz_result <- mfuzz_analysis(celltype = celltype,cluster_num = cn,showDF=FALSE,save = TRUE,showPic = FALSE)
    mfuzz_cluster <- mfuzz_result[[2]]
    #paste0("./mfuzz_cluster/",celltype,'/mfuzz_',celltype,'_cl',cn,'_origin.rds') %>% saveRDS(mfuzz_cluster,file=.)
    #paste0("./mfuzz_cluster/",celltype,'/mfuzz_',celltype,'_cl',cn,'_origin.rds  SAVED!') %>% print()
}

In [ ]:
mfuzz_geneOrder(mfuzz_result,celltype = celltype,pearson.prop = 0.4,showDF = FALSE,save=TRUE) -> final_result